In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from gensim.models import Word2Vec
import warnings 
warnings.filterwarnings(action = 'ignore') 

In [2]:
pageview = pd.read_csv('pageview.csv', error_bad_lines = False)
pageview.shape

b'Skipping line 1322258: expected 4 fields, saw 5\n'


(2749999, 4)

In [3]:
pageview.head()

,ROW_NUM,USER_ID,URL_PATH,CENSUS_KEY
0,2750000,5150228a-ad62-4a14-9169-79e7979e1a5e_46ac5dea-...,/en/friends-feelings/common-challenges/self-co...,58a9cf6a144233ad9c286a2533872b2c3a9c1998
1,2749999,5150228a-ad62-4a14-9169-79e7979e1a5e_f13c5ba7-...,/en/school-learning/partnering-with-childs-sch...,79cfc7a3c90fa4fc002ec3978716f823078afc23
2,2749998,5150228a-ad62-4a14-9169-79e7979e1a5e_22e91c69-...,/en/learning-attention-issues/child-learning-d...,bc2e78278714e173c09535d7d0260e6299b9bd09
3,2749997,5150228a-ad62-4a14-9169-79e7979e1a5e_5fe37e08-...,/en/learning-attention-issues/child-learning-d...,07e692d47d057cb1d0409cf6e701f9be5cda3e36
4,2749996,5150228a-ad62-4a14-9169-79e7979e1a5e_f106f359-...,/en/learning-attention-issues/child-learning-d...,85e32c288526fc2b24b0db665254c7f2b433a14e


In [4]:
census = pd.read_csv('census.csv', error_bad_lines = False)
census.head()

,NORTH_SOUTH,CENSUS_KEY,TIMEZONE,DIFF_FROM_US_MEAN_INCOME,PCT_TOTAL_URBAN_HH,PCT_TOTAL_RURAL_HH,PCT_BELOW_POVERTY_LEVEL,PCT_UNDER_5_YRS_AGE,PCT_SPEAKS_LANGUAGE_OTHER_THAN_ENGLISH,AVG_HH_SIZE,...,PCT_TOTAL_35_TO_44_BACHELORS,PCT_TOTAL_45_TO_64_BACHELORS,PCT_TOTAL_65_AND_ABOVE_BACHELORS,PCT_TOTAL_HISP_OR_LATINX,PCT_TOTAL_WHITE_NOT_HISP_OR_LATINX,PCT_TOTAL_WHITE_ALONE,PCT_TOTAL_BLACK_OR_AA_ALONE,PCT_TOTAL_AMERICAN_INDIAN_AND_ALASKA_NATIVE_ALONE,PCT_TOTAL_ASIAN_ALONE,PCT_TOTAL_NATIVE_HI_AND_PACIFIC_ISLANDER_ALONE
0,NORTH,026fadfabedc8fb74b6541384357664505c7ce45,America/New York,-7187,0.205,-0.205,-5.3,-1.4,-4.5,-0.35,...,0.011,0.004,0.004,-0.110,0.264,0.197,-0.114,-0.008,-0.023,0.000
1,NORTH,bb0fce0a981352d43b40f5b844859ebdc246c705,America/New York,-979,0.069,-0.069,16.7,-3.2,-3.4,-0.13,...,-0.003,0.044,0.04,-0.113,0.119,0.045,-0.071,-0.002,0.063,-0.001
2,NORTH,423954bfe8bae9f4faa66eda54301e9aebff4c9a,America/New York,-28,-0.79,0.79,-7.7,1,-16.4,0,...,-0.008,-0.013,-0.019,-0.150,0.345,0.252,-0.121,-0.008,-0.052,-0.002
3,NORTH,c6a78684b871baa185bd835e6a5ee14790d04ad5,America/New York,-4524,0.092,-0.092,-4.6,-2.7,-12.6,-0.37,...,0.004,0.04,0.008,-0.123,0.275,0.196,-0.104,-0.008,-0.036,-0.002
4,NORTH,bce53624912fa698b3df3b3a7a11bf9ba6a0eeb3,America/New York,19681,0.185,-0.185,-10.1,-1.5,-12.9,0,...,0.01,0.058,0.027,-0.153,0.254,0.159,-0.077,-0.007,-0.008,-0.002


In [5]:
content = pd.read_csv('content.csv', error_bad_lines = False)
content.head()

,url,entry_type,title,article_content,meta_title,meta_description,meta_keywords
0,/g26ftzdilh/bfunderstanding-dyslexia,Deep Dive Article,Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Understanding Dyslexia in Children | Dyslexia ...,Have you asked yourself what is dyslexia? Lear...,"dyslexia, what is dyslexia, signs of dyslexia,..."
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,Deep Dive Article,Blazingly Fast Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Entender la dislexia | ¿Qué es la discapacidad...,La dislexia es una condición real del cerebro ...,"Definición de la dislexia, qué es la dislexia,..."
2,/learning-attention-issues/getting-started/wha...,Deep Dive Article,Awareness Month 2018: Supporting the 1 in 5,When it comes to learning and attention issues...,Awareness Month 2018: Supporting the 1 in 5,Understood is celebrating ADHD and learning di...,"learning disability awareness, adhd awareness,..."
3,/learning-attention-issues/getting-started/wha...,Deep Dive Article,"Helping Your Child, and You, #BeUnderstood","October is Learning Disabilities, ADHD, and Dy...","Helping Your Child, and You, #BeUnderstood",When kids with learning and attention issues a...,"BeUnderstood, #BeUnderstood, Be Understood, Un..."
4,/learning-attention-issues/child-learning-disa...,Deep Dive Article,Understanding ADHD,What is ADHD? ADHD is a common condition that ...,What Is ADHD,"What is ADHD, and how does it affect kids? Use...","adhd, add, what is adhd, whats adhd, define ad..."


In [6]:
# select features from original dataset
import string
cotent_texts = content[['title', 'article_content', 'meta_title', 'meta_description', 'meta_keywords']]
content['merged'] = cotent_texts.astype(str).apply(' '.join, 1).str.lower()
content['merged'] = content['merged'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
content['merged_words'] = content['merged'].apply(lambda x: x.split(' '))
content.head()

,url,entry_type,title,article_content,meta_title,meta_description,meta_keywords,merged,merged_words
0,/g26ftzdilh/bfunderstanding-dyslexia,Deep Dive Article,Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Understanding Dyslexia in Children | Dyslexia ...,Have you asked yourself what is dyslexia? Lear...,"dyslexia, what is dyslexia, signs of dyslexia,...",understanding dyslexia raising a child with dy...,"[understanding, dyslexia, raising, a, child, w..."
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,Deep Dive Article,Blazingly Fast Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Entender la dislexia | ¿Qué es la discapacidad...,La dislexia es una condición real del cerebro ...,"Definición de la dislexia, qué es la dislexia,...",blazingly fast understanding dyslexia raising ...,"[blazingly, fast, understanding, dyslexia, rai..."
2,/learning-attention-issues/getting-started/wha...,Deep Dive Article,Awareness Month 2018: Supporting the 1 in 5,When it comes to learning and attention issues...,Awareness Month 2018: Supporting the 1 in 5,Understood is celebrating ADHD and learning di...,"learning disability awareness, adhd awareness,...",awareness month 2018 supporting the 1 in 5 whe...,"[awareness, month, 2018, supporting, the, 1, i..."
3,/learning-attention-issues/getting-started/wha...,Deep Dive Article,"Helping Your Child, and You, #BeUnderstood","October is Learning Disabilities, ADHD, and Dy...","Helping Your Child, and You, #BeUnderstood",When kids with learning and attention issues a...,"BeUnderstood, #BeUnderstood, Be Understood, Un...",helping your child and you beunderstood octobe...,"[helping, your, child, and, you, beunderstood,..."
4,/learning-attention-issues/child-learning-disa...,Deep Dive Article,Understanding ADHD,What is ADHD? ADHD is a common condition that ...,What Is ADHD,"What is ADHD, and how does it affect kids? Use...","adhd, add, what is adhd, whats adhd, define ad...",understanding adhd what is adhd adhd is a comm...,"[understanding, adhd, what, is, adhd, adhd, is..."


In [7]:
"""
size: The number of dimensions of the embeddings and the default is 100.
window: The maximum distance between a target word and words around the target word. The default window is 5.
min_count: The minimum count of words to consider when training the model; 
           words with occurrence less than this count will be ignored. 
           The default for min_count is 5.
workers: The number of partitions during training and the default workers is 3.
sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.
"""

word2vec = Word2Vec(content['merged_words'], min_count=3, size=32, workers=3, window=3, sg=0) # CBOW
print(word2vec.similarity('dyslexia', 'adhd'))
print(word2vec.similarity('dyslexia', 'the'))

0.7107106
0.01268132


In [8]:
word_list = list(word2vec.wv.vocab.keys())
word_embeddings = pd.Series(word_list, index=word_list).apply(lambda x: word2vec[x])
df = pd.DataFrame.from_dict(word_embeddings.to_dict()).T
colnames = ['embed_' + str(i) for i in range(32)]
df.columns = colnames
df

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_22,embed_23,embed_24,embed_25,embed_26,embed_27,embed_28,embed_29,embed_30,embed_31
understanding,1.041232,0.134059,1.544623,2.261670,1.170518,1.690798,-1.750672,-0.327919,0.492712,-0.388972,...,-0.130826,-1.210857,-0.209920,0.187149,0.118186,-0.499906,0.157473,1.904447,-0.470246,2.123954
dyslexia,0.000348,0.524082,1.890136,-0.434893,0.547984,0.403794,-3.303700,2.716385,0.080651,-1.380912,...,-2.536975,-1.109439,1.500149,-1.968208,-1.707113,-0.121188,2.646015,1.577660,-3.442770,-0.315095
raising,0.037151,-0.179233,0.602802,0.343373,-0.448969,0.970694,-0.651955,0.556306,-0.044394,-0.305125,...,-0.976115,-0.834627,-0.094196,-0.175323,-0.300789,0.044132,-0.156259,0.246829,0.160962,0.144420
a,0.233941,-1.137978,0.623627,-1.386728,1.191788,-2.158212,-2.728078,0.532340,-2.680474,1.645256,...,-1.936051,0.768852,-0.203053,0.991834,-0.875694,-1.040371,-0.961814,0.673134,-1.448818,-0.490537
child,1.372215,-0.094236,2.337813,0.961604,2.279577,-0.588124,1.959074,1.956799,0.576694,1.221735,...,-0.781837,0.530672,-2.996617,4.223913,0.748176,3.038354,5.048384,2.292927,-1.420201,-0.321425
with,-1.142551,-0.094780,0.031750,2.811828,-1.558127,1.790765,-0.352777,2.007046,-1.311177,0.372024,...,-1.527027,-2.097253,-0.622706,-1.396681,-1.288497,0.821580,1.277750,1.862940,1.074858,2.312098
can,-2.518736,1.689059,0.748537,3.470533,-1.928903,-1.604759,0.702085,2.255016,2.385264,-1.618224,...,-0.177036,1.038162,1.583721,2.533094,-1.896990,1.425657,-0.529686,3.518937,1.466256,1.084285
stir,0.004463,-0.253834,-0.129564,0.033437,-0.028144,0.014941,0.060695,0.104774,-0.036922,0.154049,...,-0.133868,-0.060330,-0.011165,0.167208,-0.065357,-0.047948,-0.127301,0.069351,-0.140536,-0.007980
up,0.863519,-0.790742,-3.401620,-0.706783,-2.273785,1.822709,-0.248584,3.162732,-1.691932,1.514426,...,-1.133339,0.131746,-0.640544,2.295852,1.274200,0.804221,-2.332276,2.155548,-4.004697,0.533413
lot,-0.728298,-1.116093,1.647210,2.750019,-1.088318,-0.104989,1.922449,1.628295,-3.269247,-0.086909,...,0.045861,-0.483865,-0.197716,-0.849978,0.115797,-0.321750,3.761805,1.412348,-0.591592,-0.565490
